In [102]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [103]:
data=pd.read_csv('../input/titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [104]:
data.skew()

PassengerId    0.000000
Survived       0.478523
Pclass        -0.630548
Age            0.389108
SibSp          3.695352
Parch          2.749117
Fare           4.787317
dtype: float64

In [105]:
data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [106]:
data['Sex']=data['Sex'].replace(['male','female'],[1,0])
print(data['Sex'].value_counts())

1    577
0    314
Name: Sex, dtype: int64


In [107]:
#creating another feature
#treating sex feature as male and creating another feature female
data['Female']=data['Sex']^1
print(data['Female'].value_counts())

0    577
1    314
Name: Female, dtype: int64


In [108]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [109]:
#removing nan values from Age and Embarked columns
data['Age']=data['Age'].replace(np.nan,data['Age'].median())
data['Embarked']=data['Embarked'].replace(np.nan,'S')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
 8   Female    891 non-null    int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 62.8+ KB


In [110]:
data['Embarked']=data['Embarked'].replace(['S','C','Q'],[1,2,3])
print(data['Embarked'].value_counts())

1    646
2    168
3     77
Name: Embarked, dtype: int64


In [111]:
#creating different columns for each port of embarkation
data['C']=data['Embarked']^1
data['C']=data['C'].replace([2,3],[0,1])
print(data['C'].value_counts())

0    723
1    168
Name: C, dtype: int64


In [112]:
data['Q']=data['Embarked']^1
data['Q']=data['Q'].replace([3,2],[0,1])
print(data['Q'].value_counts())

0    814
1     77
Name: Q, dtype: int64


In [113]:
#treating embarked column as port of embarkation=S
data['Embarked']=data['Embarked'].replace([2,3],[0,0])
print(data['Embarked'].value_counts())

1    646
0    245
Name: Embarked, dtype: int64


In [114]:
data['Pclass'].value_counts()
#treating pclass as pclass1

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [115]:
#creating different feature for each pclass
data['pclass2']=data['Pclass'].replace([3,1,2],[0,0,1])
data['pclass2'].value_counts()


0    707
1    184
Name: pclass2, dtype: int64

In [116]:
data['pclass3']=data['Pclass'].replace([1,3,2],[0,1,0])
data['pclass3'].value_counts()

1    491
0    400
Name: pclass3, dtype: int64

In [117]:
#removing value of all different pclass from pclass column as it now contains only pclass1
data['Pclass']=data['Pclass'].replace([2,3],0)
data['Pclass'].value_counts()

0    675
1    216
Name: Pclass, dtype: int64

In [118]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Female,C,Q,pclass2,pclass3
0,0,0,1,22.0,1,0,7.2500,1,0,0,0,0,1
1,1,1,0,38.0,1,0,71.2833,0,1,1,0,0,0
2,1,0,0,26.0,0,0,7.9250,1,1,0,0,0,1
3,1,1,0,35.0,1,0,53.1000,1,1,0,0,0,0
4,0,0,1,35.0,0,0,8.0500,1,0,0,0,0,1


In [119]:
y=pd.DataFrame(data['Survived'])
data.drop(['Survived'],axis=1,inplace=True)

In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
 7   Female    891 non-null    int64  
 8   C         891 non-null    int64  
 9   Q         891 non-null    int64  
 10  pclass2   891 non-null    int64  
 11  pclass3   891 non-null    int64  
dtypes: float64(2), int64(10)
memory usage: 83.7 KB
